In [1]:
#install cntk and required library
import os,sys
import numpy as np
import cntk as C
from cntk import load_model, combine
import cntk.io.transforms as xforms
from cntk.logging import graph
from cntk.logging.graph import get_node_outputs
import zipfile
import shutil
import glob
import sklearn as sk
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
import pickle as pk
import xgboost as xg
import time
# import tensorflow as tf

In [2]:
# dataframe with features from resnet 18 one layer before last layer for train dataset
df = pd.read_csv("predOutput_1000.txt", delim_whitespace=True, header=None)
df_cls = pd.read_csv("image-location-with-class.txt", delim_whitespace=True, header=None)
df_cls.columns = ['location','class']
df = df.join(df_cls['class'])


In [3]:

# dataframe with features from resnet 18 one layer before last layer for val dataset
df_validate = pd.read_csv("predOutput_val_1000.txt", delim_whitespace=True, header=None)
df_validate_cls = pd.read_csv("image-location-val.txt", delim_whitespace=True, header=None)
df_validate_cls.columns = ['location','class']
df_validate = df_validate.join(df_validate_cls['class'])
print(df_validate)

             0         1         2         3         4          5         6  \
0    -1.066392 -0.318509 -1.664783 -0.264905 -1.684140   0.434602 -2.551604   
1     3.013852 -0.940698  2.668768  2.740596  0.025980   3.220846 -3.906793   
2    -1.799187 -2.897550 -2.906348 -2.983232 -4.415853  -0.146917 -6.959669   
3     0.827173 -0.219210 -2.562124 -2.250758 -4.175933   0.834077 -1.877497   
4    -0.022826  0.932449 -6.492920 -5.550576 -4.012052   0.208040 -4.701711   
5    -0.549076 -0.036998 -3.732693  0.917979 -1.922013   1.015206 -4.432506   
6    -1.305109  1.935297 -5.052127 -3.319174 -4.129058  -0.380575 -4.448411   
7     3.441793  0.227624 -0.673989  0.550912  0.620432   1.769606 -1.498477   
8     1.607878  1.756927 -3.194895  0.024584 -0.332034   2.895038 -1.152025   
9    -1.935967  0.796424 -2.717592 -2.859518 -2.734782   1.435592 -2.617879   
10    2.195558 -2.179096 -0.896631 -0.572897 -0.960225   0.586332 -3.555867   
11    1.118002  1.846784 -3.646180 -4.369446 -4.0068

In [ ]:
from sklearn.model_selection import train_test_split
X_train = df.drop('class', axis=1).values
y_train = df['class'].values.astype(np.integer)
# X_train, X_test, y_train, y_test = train_test_split(X,y,train_size = 0.5,stratify=y) 
svm = SVC(kernel='rbf')
print(y_train)
# svm.fit(X_train,y_train)



In [ ]:
with open ("svm.pkl",'wb') as output:
    pk.dump(svm,output,pk.HIGHEST_PROTOCOL)


In [ ]:
svm_pred = svm.predict(X)

In [4]:
X_train = df.drop('class', axis=1).values
y_train = df['class'].values.astype(np.integer)

In [ ]:
start_time_rf = time.time()
random_forest_clf = RandomForestClassifier(n_estimators=15,random_state=0)
random_forest_clf.fit(X_train,y_train)
total_time_rf = start_time_rf - time.time()


In [ ]:
print(total_time_rf)

In [ ]:
with open ("random_forest.pkl",'wb') as output:
    pk.dump(random_forest_clf,output,pk.HIGHEST_PROTOCOL)

In [5]:
X_pred = df_validate.drop('class', axis=1).values
y_pred_truth = df_validate['class'].values.astype(np.integer)

In [ ]:
y_pred_rf = random_forest_clf.predict(X_pred)

In [6]:
from sklearn.metrics import accuracy_score

In [ ]:
random_forest_accuracy = accuracy_score(y_true=y_pred_truth, y_pred=y_pred_rf )
print(random_forest_accuracy)

In [ ]:
start_time_xg = time.time()
xgboost_clf = xg.XGBClassifier(n_estimators=100)
xgboost_clf.fit(X_train, y_train)


In [ ]:
total_time_xg = start_time_xg - time.time()
print(total_time_xg)

In [ ]:
with open ("xgBoost.pkl",'wb') as output:
    pk.dump(xgboost_clf,output,pk.HIGHEST_PROTOCOL)

In [ ]:
y_pred_xg = xgboost_clf.predict(X_pred)

In [ ]:
xg_accuracy = accuracy_score(y_true=y_pred_truth, y_pred=y_pred_rf )
print(xg_accuracy)

In [ ]:
def get_feature_class(dataframe):
    cls_loc = df = pd.read_csv("map_clsloc.txt", delim_whitespace=True, header=None)


In [ ]:
p_df = p

In [ ]:
try:
    from urllib.request import urlretrieve 
except ImportError: 
    from urllib import urlretrieve
    
    
# # download Microsoft grocery data for object detection

def download_grocery_data():
#     base_folder = os.path.dirname(os.path.abspath(__file__))
#     dataset_folder = os.path.join(base_folder, "..")
#     dataset_folder = os.path.join("..")
    if not os.path.exists(os.path.join("Grocery", "testImages")):
        filename = os.path.join("Grocery.zip")
        if not os.path.exists(filename):
            url = "https://www.cntk.ai/DataSets/Grocery/Grocery.zip"
            print('Downloading data from ' + url + '...')
            urlretrieve(url, filename)
            
        try:
            print('Extracting ' + filename + '...')
            with zipfile.ZipFile(filename) as myzip:
                myzip.extractall()
#             if platform != "win32":
                testfile  = os.path.join("Grocery", "test.txt")
                unixfile = os.path.join("Grocery", "test_unix.txt")
                out = open(unixfile, 'w')
                with open(testfile) as f:
                    for line in f:
                        out.write(line.replace('\\', '/'))
                out.close()
                shutil.move(unixfile, testfile)
        finally:
            os.remove(filename)
        print('Done.')
    else:
        print('Data already available at ' + '/Grocery')
    

download_grocery_data()

In [ ]:
# download Tiny Imagenet(Stanford CS231N) (Zipped Size = 236MB)

def download_tinyImagenet_data():
#     base_folder = os.path.dirname(os.path.abspath(__file__))
#     dataset_folder = os.path.join(base_folder, "..")
#     dataset_folder = os.path.join("..")
    if not os.path.exists(os.path.join("tiny-imageset", "train")):
        filename = os.path.join("tiny-imagenet-200.zip")
        if not os.path.exists(filename):
            url = "http://www.image-net.org/image/tiny/tiny-imagenet-200.zip"
            print('Downloading data from ' + url + '...')
            urlretrieve(url, filename)
            
        try:
            print('Extracting ' + filename + '...')
            with zipfile.ZipFile(filename) as myzip:
                myzip.extractall()
#             if platform != "win32":
                testfile  = os.path.join("tiny-imageset", "test.txt")
                unixfile = os.path.join("tiny-imageset", "test_unix.txt")
                out = open(unixfile, 'w')
                with open(testfile) as f:
                    for line in f:
                        out.write(line.replace('\\', '/'))
                out.close()
                shutil.move(unixfile, testfile)
        finally:
            os.remove(filename)
        print('Done.')
    else:
        print('Data already available at ' + '/tiny-imageset')
    

download_tinyImagenet_data()

In [ ]:
df_cls = pd.read_csv("map_clsloc.txt", delim_whitespace=True, header=None)
print(df_cls)

In [ ]:
# writes image location is test.text file which will be fed to model to extract features

def write_image_location_in_text_file():
    myvar = glob.glob('./tiny-imagenet-200/train/**/*.JPEG',recursive=True)
    feature_vector_file = os.path.join("tiny-imagenet-200","image-location.txt")
    with open(feature_vector_file, 'wb') as f:
        for url in myvar:
#             dir_cls = os.path.basename(os.path.dirname(os.path.dirname(url)))
#             df_cls_val = df_cls[df_cls[0].str.contains(dir_cls)]
#             cls_val = str(df_cls_val.iloc[0][1]).encode('utf-8',errors='ignore')
#           cls_dir = os.path.basename(os.dirname(main_dir))
#             print(cls_dir)
#             print(url)
            f.write(url.encode()+"\t".encode()+"0".encode()+"\n".encode())
            
#             np.savetxt(feature_vector_file, url, fmt="%s")

                               
write_image_location_in_text_file()

In [ ]:
try:
    from urllib.request import urlretrieve 
except ImportError: 
    from urllib import urlretrieve

# Add models here like this: (category, model_name, model_url)
models = (('Image Classification', 'AlexNet_ImageNet_CNTK', 'https://www.cntk.ai/Models/CNTK_Pretrained/AlexNet_ImageNet_CNTK.model'),
          ('Image Classification', 'AlexNet_ImageNet_Caffe', 'https://www.cntk.ai/Models/Caffe_Converted/AlexNet_ImageNet_Caffe.model'),
          ('Image Classification', 'InceptionV3_ImageNet_CNTK', 'https://www.cntk.ai/Models/CNTK_Pretrained/InceptionV3_ImageNet_CNTK.model'),
          ('Image Classification', 'BNInception_ImageNet_Caffe', 'https://www.cntk.ai/Models/Caffe_Converted/BNInception_ImageNet_Caffe.model'),
          ('Image Classification', 'ResNet18_ImageNet_CNTK', 'https://www.cntk.ai/Models/CNTK_Pretrained/ResNet18_ImageNet_CNTK.model'),
          ('Image Classification', 'ResNet34_ImageNet_CNTK', 'https://www.cntk.ai/Models/CNTK_Pretrained/ResNet34_ImageNet_CNTK.model'),
          ('Image Classification', 'ResNet50_ImageNet_CNTK', 'https://www.cntk.ai/Models/CNTK_Pretrained/ResNet50_ImageNet_CNTK.model'),
          ('Image Classification', 'ResNet20_CIFAR10_CNTK', 'https://www.cntk.ai/Models/CNTK_Pretrained/ResNet20_CIFAR10_CNTK.model'),
          ('Image Classification', 'ResNet110_CIFAR10_CNTK', 'https://www.cntk.ai/Models/CNTK_Pretrained/ResNet110_CIFAR10_CNTK.model'),
          ('Image Classification', 'ResNet50_ImageNet_Caffe', 'https://www.cntk.ai/Models/Caffe_Converted/ResNet50_ImageNet_Caffe.model'),
          ('Image Classification', 'ResNet101_ImageNet_Caffe', 'https://www.cntk.ai/Models/Caffe_Converted/ResNet101_ImageNet_Caffe.model'),
          ('Image Classification', 'ResNet152_ImageNet_Caffe', 'https://www.cntk.ai/Models/Caffe_Converted/ResNet152_ImageNet_Caffe.model'),
          ('Image Classification', 'VGG16_ImageNet_Caffe', 'https://www.cntk.ai/Models/Caffe_Converted/VGG16_ImageNet_Caffe.model'),
          ('Image Classification', 'VGG19_ImageNet_Caffe', 'https://www.cntk.ai/Models/Caffe_Converted/VGG19_ImageNet_Caffe.model'),
          ('Image Object Detection', 'Fast-RCNN_grocery100', 'https://www.cntk.ai/Models/FRCN_Grocery/Fast-RCNN_grocery100.model'),
          ('Image Object Detection', 'Fast-RCNN_Pascal', 'https://www.cntk.ai/Models/FRCN_Pascal/Fast-RCNN.model'))

def download_model(model_file_name, model_url):
#     model_dir = os.path.dirname(os.path.abspath(__file__))
#     filename = os.path.join(model_dir, model_file_name)
    filename = os.path.join(model_file_name)

    if not os.path.exists(filename):
        print('Downloading model from ' + model_url + ', may take a while...')
        urlretrieve(model_url, filename)
        print('Saved model as ' + filename)
    else:
        print('CNTK model already available at ' + filename)
    
def download_model_by_name(model_name):
    if model_name.endswith('.model'):
        model_name = model_name[:-6]

    model = next((x for x in models if x[1]==model_name), None)
    if model is None:
        print("ERROR: Unknown model name '%s'." % model_name)
        list_available_models()
    else:
        download_model(model_name + '.model', model[2])

def list_available_models():
    print("\nAvailable models (for more information see Readme.md):")
    max_cat = max(len(x[1]) for x in models)
    max_name = max(len(x[1]) for x in models)
    print("{:<{width}}   {}".format('Model name', 'Category', width=max_name))
    print("{:-<{width}}   {:-<{width_cat}}".format('', '', width=max_name, width_cat=max_cat))
    for model in sorted(models):
        print("{:<{width}}   {}".format(model[1], model[0], width=max_name))

In [ ]:
# base_folder = os.path.dirname(os.path.abspath(__file__))

# sys.path.append(os.path.join(base_folder, "..", "DataSets", "Grocery"))
sys.path.append(os.path.join( "..", "DataSets", "Grocery"))

# from install_grocery import download_grocery_data
# download_grocery_data()
   
# sys.path.append(os.path.join(base_folder, "..", "..", "..", "PretrainedModels"))
sys.path.append(os.path.join("..", "..", "..", "PretrainedModels"))

# from download_model import download_model_by_name
download_model_by_name("ResNet18_ImageNet_CNTK")
# download_model_by_name("InceptionV3_ImageNet_CNTK")
# download_model_by_name("AlexNet_ImageNet_CNTK")
download_model_by_name("ResNet152_ImageNet_Caffe")

In [ ]:
def create_mb_source(image_height, image_width, num_channels, map_file):
    transforms = [xforms.scale(width=image_width, height=image_height, channels=num_channels, interpolations='linear')]
    return C.io.MinibatchSource(
        C.io.ImageDeserializer(map_file, C.io.StreamDefs(
            features=C.io.StreamDef(field='image', transforms=transforms),
            labels=C.io.StreamDef(field='label', shape=1000))),
        randomize=False)

def eval_and_write(model_file, node_name, output_file, minibatch_source, num_objects):
    # load model and pick desired node as output
    loaded_model  = load_model(model_file)
    node_in_graph = loaded_model.find_by_name(node_name)
    output_nodes  = combine([node_in_graph.owner])

    # evaluate model and get desired node output
    print("Evaluating model for output node %s" % node_name)
    features_si = minibatch_source['features']
    with open(output_file, 'wb') as results_file:
        for i in range(0, num_objects):
            mb = minibatch_source.next_minibatch(1)
            output = output_nodes.eval(mb[features_si])

            # write results to file
            out_values = output[0].flatten()
            np.savetxt(results_file, out_values[np.newaxis], fmt="%.6f")

def main():
    # define location of model and data and check existence
#     base_folder = os.path.dirname(os.path.abspath(__file__))
#     model_file  = os.path.join(base_folder, "..", "..", "..", "PretrainedModels", "ResNet18_ImageNet_CNTK.model")
#     model_file  = os.path.join("..", "..", "..", "PretrainedModels", "ResNet18_ImageNet_CNTK.model")
    model_file  = os.path.join("ResNet18_ImageNet_CNTK.model")
    if(os.path.exists(model_file)):
        print("model exists")
    
#     print(model_file)


#     map_file    = os.path.join(base_folder, "..", "DataSets", "Grocery", "test.txt")
#     map_file    = os.path.join("..", "DataSets", "Grocery", "test.txt")
    map_file = os.path.join("tiny-imagenet-200", "image-location.txt")
#     print(map_file)

#     os.chdir(os.path.join(base_folder, "..", "DataSets", "Grocery"))
#     os.chdir(os.path.join("..", "Grocery"))

#     os.chdir(os.path.join("..", "DataSets", "Grocery"))

#     image_file = os.path.join("Grocery", "testImages","WIN_20160803_11_28_42_Pro.jpg")
#     if not (os.path.exists(model_file)):
#         print("model bhetena")
        
#     if(os.path.exists(image_file)):
#         print("image bhetyo")
        
#     if not (os.path.exists(map_file)):
#         print("map file bhetena")

#     if not (os.path.exists(model_file) and os.path.exists(map_file)):
#         print("Please run 'python install_data_and_model.py' first to get the required data and model.")
#         exit(0)

    # create minibatch source
    image_height = 224
    image_width  = 224
    num_channels = 3
    minibatch_source = create_mb_source(image_height, image_width, num_channels, map_file)

    # use this to print all node names of the model (and knowledge of the model to pick the correct one)
    node_outputs = get_node_outputs(load_model(model_file))
#     for out in node_outputs: print("{0} {1}".format(out.name, out.shape))

    # use this to get 1000 class predictions (not yet softmaxed!)
    node_name = "z"
    output_file = os.path.join("predOutput_val.txt")
#     output_file = os.path.join("fish.txt")

    # use this to get 512 features from the last but one layer of ResNet_18
#     node_name = "z.x"
#     output_file = os.path.join(base_folder, "layerOutput.txt")
#     output_file = os.path.join("layerOutput.txt")
    
    # evaluate model and write out the desired layer output
    eval_and_write(model_file, node_name, output_file, minibatch_source, num_objects=99999)

    print("Done. Wrote output to %s" % output_file)

main()

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
print(total_time_nn)

In [ ]:
with open ("neural_network.pkl",'wb') as output:
    pk.dump(nn_clf,output,pk.HIGHEST_PROTOCOL)

In [ ]:
y_pred_nn = nn_clf.predict(X_pred)

nn_accuracy_score = accuracy_score(y_true=y_pred_truth, y_pred=y_pred_nn )


In [ ]:
print(nn_accuracy_score)